In [1]:
from nilearn import connectome
import numpy as np
import os
import math
import pickle
from sklearn import cluster, metrics
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from esfMRI import sliceWindows, plot_sates, cluster_save_states, clustering_evaluate, windows_evaluate, align, get_missing_state

In [2]:
with open("time_series2.pkl", "rb") as f:
    data = pickle.load(f)

In [3]:
subid = "sub-314"

In [4]:
mpl.use("agg")
window_length_Second = [120, 150, 180]
sliding_steps = [1, 6, 12, 18]
ks = list(range(2, 15))

In [9]:
# 评估指定状态数时窗口尺寸的影响
save_dir_preop = f"cluster_evaluate/windows_length/{subid}/preop/"
save_dir_postop = f"cluster_evaluate/windows_length/{subid}/postop/"
os.makedirs(save_dir_preop, exist_ok=True)
os.makedirs(save_dir_postop, exist_ok=True)
for step in sliding_steps:
    windows_evaluate(data, subid, range(100, 181, 10), step, 3, save_dir_preop)
    # clustering_evaluate(windows_postop, ks, save_path_postop)

In [6]:
# 评估不同尺寸和不同步长下滑动窗口效果
save_dir_preop = f"cluster_evaluate/diff_steps/{subid}/preop/"
save_dir_postop = f"cluster_evaluate/diff_steps/{subid}/postop/"
os.makedirs(save_dir_preop, exist_ok=True)
os.makedirs(save_dir_postop, exist_ok=True)
for time in window_length_Second:
    for step in sliding_steps:
        windows_preop = []
        for run, items in data[subid]["ses-preop"].items():
            preopTR = math.ceil(time/items["TR"])
            stepTR = math.ceil(step/items["TR"])
            windows_preop += sliceWindows(items["time_series"], preopTR, stepTR)
        windows_postop = []
        for run, items in data[subid]["ses-postop"].items():
            postopTR = math.ceil(time/items["TR"])
            stepTR = math.ceil(step/items["TR"])
            windows_postop += sliceWindows(items["time_series"], postopTR, stepTR)
        os.makedirs(f"{save_dir_preop}/{time}s", exist_ok=True)
        os.makedirs(f"{save_dir_postop}/{time}s", exist_ok=True)
        save_path_preop = f"{save_dir_preop}/{time}s/{step}s.png"
        save_path_postop = f"{save_dir_postop}/{time}s/{step}s.png"
        if len(windows_preop) > 2:
            if not os.path.exists(save_path_preop):
                clustering_evaluate(windows_preop, ks, save_path_preop)
        if len(windows_postop) > 2:
            if not os.path.exists(save_path_postop):
                clustering_evaluate(windows_postop, ks, save_path_postop)
        plt.cla()
        plt.clf()
        plt.close("all")

In [7]:
# group average
align_length = 130
save_dir_preop = f"cluster_evaluate/average/{subid}/preop/"
save_dir_postop = f"cluster_evaluate/average/{subid}/postop/"
os.makedirs(save_dir_preop, exist_ok=True)
os.makedirs(save_dir_postop, exist_ok=True)
time_series_preop = None
time_series_postop = None
count = 0
for run, items in data[subid]["ses-preop"].items():
    if items["time_series"].shape[0] < align_length:
        continue
    count += 1
    time_series_preop = align(items["time_series"], align_length) if time_series_preop is None else time_series_preop + align(items["time_series"], align_length)
time_series_preop = time_series_preop/count
align_length = 200
count = 0
for run, items in data[subid]["ses-postop"].items():
    if items["time_series"].shape[0] < align_length:
        continue
    count += 1
    time_series_postop = align(items["time_series"], align_length) if time_series_postop is None else time_series_postop + align(items["time_series"], align_length)
time_series_postop = time_series_postop/count


In [8]:
# group average evaluate
for time in window_length_Second:
    preopTR = math.ceil(time/data[subid]["ses-preop"]["run-01"]["TR"])
    postopTR = math.ceil(time/data[subid]["ses-postop"]["run-01"]["TR"])
    for step in sliding_steps:
        windows_preop = sliceWindows(time_series_preop, preopTR, step)
        windows_postop = sliceWindows(time_series_postop, postopTR, step)
        os.makedirs(f"{save_dir_preop}/{time}s", exist_ok=True)
        os.makedirs(f"{save_dir_postop}/{time}s", exist_ok=True)
        save_path_preop = f"{save_dir_preop}/{time}s/{step}s.png"
        save_path_postop = f"{save_dir_postop}/{time}s/{step}s.png"
        if len(windows_preop) > 2:
            if not os.path.exists(save_path_preop):
                clustering_evaluate(windows_preop, ks, save_path_preop)
        if len(windows_postop) > 2:
            if not os.path.exists(save_path_postop):
                clustering_evaluate(windows_postop, ks, save_path_postop)
        plt.cla()
        plt.clf()
        plt.close("all")

In [11]:
# 平均后聚4类，状态变化
target_states = 3
for time in window_length_Second:
    preopTR = math.ceil(time/data[subid]["ses-preop"]["run-01"]["TR"])
    postopTR = math.ceil(time/data[subid]["ses-postop"]["run-01"]["TR"])
    for step in sliding_steps:
        windows_preop = sliceWindows(time_series_preop, preopTR, step)
        windows_postop = sliceWindows(time_series_postop, postopTR, step)
        if len(windows_preop) <= target_states and len(windows_postop) <= target_states:
            continue
        save_dir = f"states/average/{subid}/{time}/{step}"
        os.makedirs(save_dir, exist_ok=True)
        cluster_save_states(windows_preop, windows_postop, target_states, save_dir)

In [ ]:
# 拼接后聚类
target_states = 3
for time in window_length_Second:
    preopTR = math.ceil(time/data[subid]["ses-preop"]["run-01"]["TR"])
    postopTR = math.ceil(time/data[subid]["ses-postop"]["run-01"]["TR"])
    for step in sliding_steps:
        windows_preop = []
        windows_postop = []
        for run, items in data[subid]["ses-preop"].items():
            windows_preop += sliceWindows(items["time_series"], preopTR, step)
        for run, items in data[subid]["ses-postop"].items():
            windows_postop += sliceWindows(items["time_series"], postopTR, step)
        save_dir = f"states/joint/{subid}/{time}/{step}"
        os.makedirs(save_dir, exist_ok=True)
        cluster_save_states(windows_preop, windows_postop, target_states, save_dir)

In [5]:
# 拼接后聚类，输出缺少的状态
target_states = 3
step = 1
for time in window_length_Second:
    preopTR = math.ceil(time/data[subid]["ses-preop"]["run-01"]["TR"])
    postopTR = math.ceil(time/data[subid]["ses-postop"]["run-01"]["TR"])
    windows_preop = []
    windows_postop = []
    for run, items in data[subid]["ses-preop"].items():
        windows_preop += sliceWindows(items["time_series"], preopTR, step)
    for run, items in data[subid]["ses-postop"].items():
        windows_postop += sliceWindows(items["time_series"], postopTR, step)
    save_dir = f"missing_state/joint/{subid}/{target_states}states/{time}/"
    os.makedirs(save_dir, exist_ok=True)
    fcs = get_missing_state(windows_preop, windows_postop, target_states, save_dir)